In [1]:
import torch
import torch.nn as nn
import numpy as np
import math
from einops.layers.torch import Rearrange
from einops import rearrange
import torch.nn.functional as F

In [2]:
## Data processing ##

## We provided two example data (data/info_ko_org.txt and data/info_ko_ko.txt) for reference. ##

## Please provide the following information in that file: 
## (1) n x 1029bp DNA sequences, n is the number of input peaks. For each peak, the central nucleotide should be sampled at the central of that peak, and the DNA sequence length should be of 1029 bp;
## (2) n x 1029bp ATAC signal values. The ATAC signal values should be from the .bigWig file, each value is at the same position of your sampled nucleotide.
## (3) Please give n x IDs for reference of each peak

## Please write your information in a .txt file, foe example:
## line 1 (ID of this peak): >chr12 135145-135516 
## line 2 (1029bp DNA): ATCGATCG ... ... TCGA
## line 3 (1029bp ATAC): 1.28971 1.11121 ... ... 0.01234
## line 4 (next peak ID): ... ...
## ... ...
## input information of n peaks should have 3*n lines

In [3]:
## If you want to simulate the KO or KI of functional elements:
## Please provid the .txt data BEFORE and AFTER the KO or KI, and compute their gene expression respectively

In [4]:
from data_processing import *
genome_dict = torch.load('../data/kmer_dict.pkl')

raw_input_data = narrowPeak_Reader('../data/info_ko_org.txt')

data_len = int(len(raw_input_data)/3)
peak_id  = list(range(data_len))
dna_in   = list(range(data_len))
atac_in  = list(range(data_len))

for i_data in range (data_len):    
    peak_id[i_data] = raw_input_data[i_data*3][0]
    dna_in[i_data]  = pre_processing(tokenizer(raw_input_data[i_data*3+1][0], 6), genome_dict)
    for i in range (len(raw_input_data[i_data*3+2])):
        raw_input_data[i_data*3+2][i]=float(raw_input_data[i_data*3+2][i])
    atac_in[i_data] = pre_pro(raw_input_data[i_data*3+2], 6)
dna_in  = torch.tensor(dna_in)
atac_in = torch.tensor(atac_in)

print(dna_in.shape, atac_in.shape)

torch.Size([44, 1024]) torch.Size([44, 1024])


In [5]:
## Specify your TSS location and strand ##
## Our example gene is GATA1, the closest ATAC peak to TSS is the 20th peak, and the gene is + strand ##
tss_loc   = 20
direction = '+'

In [6]:
## Load REformer model ##

In [7]:
from models import *
from attention import *

cuda = torch.device('cuda', 0)

dna_embed  = torch.load("../pretrained_models/dna_embed.pkl"    , map_location='cpu').to(cuda)
atac_embed = torch.load("../pretrained_models/atac_embed.pkl"   , map_location='cpu').to(cuda)
pos1_embed = torch.load("../pretrained_models/pos1_embed.pkl"   , map_location='cpu').to(cuda)
pos2_embed = torch.load("../pretrained_models/pos2_embed.pkl"   , map_location='cpu').to(cuda)
tss_embed  = torch.load("../pretrained_models/tss_embed.pkl"    , map_location='cpu').to(cuda)
pad_embed  = torch.load("../pretrained_models/pad_embed.pkl"    , map_location='cpu').to(cuda)
encoder_1  = torch.load("../pretrained_models/transformer_1.pkl", map_location='cpu').to(cuda)
encoder_2  = torch.load("../pretrained_models/transformer_2.pkl", map_location='cpu').to(cuda)
encoder_3  = torch.load("../pretrained_models/transformer_3.pkl", map_location='cpu').to(cuda)
atten_pool = torch.load("../pretrained_models/atten_pool.pkl"   , map_location='cpu').to(cuda)
ff_net     = torch.load("../pretrained_models/feedforward.pkl"  , map_location='cpu').to(cuda)

In [8]:
## compute attention score ##

In [9]:
dna_in = dna_in.to(cuda)
sig_in  = atac_in.to(cuda)

pos1 = torch.ones(129, dtype=int).to(cuda)
for i in range (len(pos1)):
    pos1[i]+=i
pos2 = torch.ones(8, dtype=int).to(cuda)
for i in range (len(pos2)):
    pos2[i]+=i
pos3 = torch.zeros(150, dtype=int).to(cuda)
pos3[tss_loc] = 0
if direction=='+':
    pos3[tss_loc-1] = 1
    pos3[tss_loc+1] = 2
    for tss_i in range (tss_loc-1):
        pos3[tss_loc-1-tss_i-1] = pos3[tss_loc-1-tss_i]+2
    for tss_i in range (dna_in.shape[0]-tss_loc-2):
        pos3[tss_loc+1+tss_i+1] = pos3[tss_loc+1+tss_i]+2
if direction=='-':
    pos3[tss_loc-1] = 2
    pos3[tss_loc+1] = 1
    for tss_i in range (tss_loc-1):
        pos3[tss_loc-1-tss_i-1] = pos3[tss_loc-1-tss_i]+2
    for tss_i in range (dna_in.shape[0]-tss_loc-2):
        pos3[tss_loc+1+tss_i+1] = pos3[tss_loc+1+tss_i]+2
        
with torch.no_grad():
        
    CLS     = dna_embed(torch.ones(dna_in.shape[0]*8, 1, dtype=int).to(cuda))
    x_POS_1 = pos1_embed(pos1)      
    x_mul   = dna_embed(dna_in.int().reshape(dna_in.shape[0]*8, 128)) + atac_embed(sig_in.int().reshape(dna_in.shape[0]*8, 128))        
    x_embed = torch.cat((CLS, x_mul), dim=1)
    x_enc_1 = encoder_1(x_embed + x_POS_1)[:,0,:].reshape(dna_in.shape[0],8,2048)
    x_POS_2 = pos2_embed(pos2)
    x_enc_2 = encoder_2(x_enc_1+x_POS_2)
    x_enc_2 = rearrange(x_enc_2, 'b n d -> b d n')
    x_enc_2 = atten_pool(x_enc_2)
    x_enc_2 = rearrange(x_enc_2, 'b d n -> b n d').squeeze(1)
    x_pad   = pad_embed(torch.zeros(150-x_enc_2.shape[0], dtype=int).to(cuda))
    x_eb3   = torch.cat((x_enc_2, x_pad), dim=0)
    x_POS_3 = tss_embed(pos3)
    x_enc_3 = encoder_3(x_eb3 + x_POS_3)
    result  = ff_net(x_enc_3).squeeze(1)

In [10]:
## print results #

In [11]:
print(result)

tensor([5.6400], device='cuda:3')


In [12]:
## Here, we additionally showed the KO of chrX_48761672_48762478 (an enhancer validated by CRISPRi)
## In the input .txt file (data/info_ko_ko.txt), we set the ATAC signal in that peak as zero

In [13]:
## Please notice that if you are performing KI of functional elements, the index of TSS location might change ##
## Please re-specify your TSS location in KI experiments ##
tss_loc   = 20

In [14]:
raw_input_data = narrowPeak_Reader('../data/info_ko_ko.txt')

data_len = int(len(raw_input_data)/3)
peak_id  = list(range(data_len))
dna_in   = list(range(data_len))
atac_in  = list(range(data_len))

for i_data in range (data_len):    
    peak_id[i_data] = raw_input_data[i_data*3][0]
    dna_in[i_data]  = pre_processing(tokenizer(raw_input_data[i_data*3+1][0], 6), genome_dict)
    for i in range (len(raw_input_data[i_data*3+2])):
        raw_input_data[i_data*3+2][i]=float(raw_input_data[i_data*3+2][i])
    atac_in[i_data] = pre_pro(raw_input_data[i_data*3+2], 6)
dna_in  = torch.tensor(dna_in)
atac_in = torch.tensor(atac_in)

print(dna_in.shape, atac_in.shape)

torch.Size([44, 1024]) torch.Size([44, 1024])


In [15]:
dna_in = dna_in.to(cuda)
sig_in  = atac_in.to(cuda)

pos1 = torch.ones(129, dtype=int).to(cuda)
for i in range (len(pos1)):
    pos1[i]+=i
pos2 = torch.ones(8, dtype=int).to(cuda)
for i in range (len(pos2)):
    pos2[i]+=i
pos3 = torch.zeros(150, dtype=int).to(cuda)
pos3[tss_loc] = 0
if direction=='+':
    pos3[tss_loc-1] = 1
    pos3[tss_loc+1] = 2
    for tss_i in range (tss_loc-1):
        pos3[tss_loc-1-tss_i-1] = pos3[tss_loc-1-tss_i]+2
    for tss_i in range (dna_in.shape[0]-tss_loc-2):
        pos3[tss_loc+1+tss_i+1] = pos3[tss_loc+1+tss_i]+2
if direction=='-':
    pos3[tss_loc-1] = 2
    pos3[tss_loc+1] = 1
    for tss_i in range (tss_loc-1):
        pos3[tss_loc-1-tss_i-1] = pos3[tss_loc-1-tss_i]+2
    for tss_i in range (dna_in.shape[0]-tss_loc-2):
        pos3[tss_loc+1+tss_i+1] = pos3[tss_loc+1+tss_i]+2
        
with torch.no_grad():
        
    CLS     = dna_embed(torch.ones(dna_in.shape[0]*8, 1, dtype=int).to(cuda))
    x_POS_1 = pos1_embed(pos1)      
    x_mul   = dna_embed(dna_in.int().reshape(dna_in.shape[0]*8, 128)) + atac_embed(sig_in.int().reshape(dna_in.shape[0]*8, 128))        
    x_embed = torch.cat((CLS, x_mul), dim=1)
    x_enc_1 = encoder_1(x_embed + x_POS_1)[:,0,:].reshape(dna_in.shape[0],8,2048)
    x_POS_2 = pos2_embed(pos2)
    x_enc_2 = encoder_2(x_enc_1+x_POS_2)
    x_enc_2 = rearrange(x_enc_2, 'b n d -> b d n')
    x_enc_2 = atten_pool(x_enc_2)
    x_enc_2 = rearrange(x_enc_2, 'b d n -> b n d').squeeze(1)
    x_pad   = pad_embed(torch.zeros(150-x_enc_2.shape[0], dtype=int).to(cuda))
    x_eb3   = torch.cat((x_enc_2, x_pad), dim=0)
    x_POS_3 = tss_embed(pos3)
    x_enc_3 = encoder_3(x_eb3 + x_POS_3)
    result  = ff_net(x_enc_3).squeeze(1)

In [16]:
print(result)

tensor([4.2302], device='cuda:3')
